#  aitextgen — Train a GPT-2 (or GPT Neo) Text-Generating Model w/ GPU

by [Max Woolf](https://minimaxir.com)

*Last updated: May 16th, 2021 (aitextgen v0.5.2)*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Colaboratory** using `aitextgen`!

For more about `aitextgen`, you can visit [this GitHub repository](https://github.com/minimaxir/aitextgen) or [read the documentation](https://docs.aitextgen.io/).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Run the cells below:


In [ ]:
!pip install -q aitextgen

In [ ]:
import aitextgen
import datetime
import gc
import logging
import os
import requests
import torch

In [ ]:
session_url = 'http://172.28.0.2:9000/api/sessions'
notebook_name = requests.get(session_url).json()[0]['name']

run_datetime = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
run_id = notebook_name + '_run_' + run_datetime

In [ ]:
log_format = '%(asctime)s — %(levelname)s — %(name)s — %(message)s'
date_format = '%d/%m/%Y %H:%M:%S'
log_level = logging.DEBUG

logging.basicConfig(format=log_format, datefmt=date_format, level=log_level)

## GPU

Colaboratory uses a Nvidia P4, an Nvidia T4, an Nvidia P100, or an Nvidia V100. For finetuning GPT-2 124M, any of these GPUs will be fine, but for text generation, a T4 or a P100 is ideal since they have more VRAM. **If you receive a T4 or a V100 GPU, you can enable `fp16=True` during training for faster/more memory efficient training.**

You can verify which GPU is active by running the cell below. If you want to try for a different GPU, go to **Runtime -> Factory Reset Runtime**.

In [ ]:
!nvidia-smi

Fri May 27 12:41:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
aitextgen.colab.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gdrive_root_dir = '/content/drive/My Drive'


## Load a Trained Model

If you already had a trained model from this notebook, running the next cell will copy the `pytorch_model.bin` and the `config.json`file from the specified folder in Google Drive into the Colaboratory VM. (If no `from_folder` is specified, it assumes the two files are located at the root level of your Google Drive)

In [ ]:
#load_model = None
load_model = 'aitextgen-CCS-9000-1000_run_2022-05-20-14-41-07/partial/articles_trainrun-5'

In [ ]:
if load_model is not None:
    model_load_dir = gdrive_root_dir + '/aitextgen/models/' + load_model
    ai = aitextgen.aitextgen(model_folder=model_load_dir, to_gpu=True)

27/05/2022 12:41:49 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/My Drive/aitextgen/models/aitextgen-CCS-9000-1000_run_2022-05-20-14-41-07/partial/articles_trainrun-5.
27/05/2022 12:41:57 — INFO — aitextgen — GPT2 loaded with 124M parameters.
27/05/2022 12:41:57 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


The next cell will allow you to load the retrained model + metadata necessary to generate text.

## Loading GPT-2 or GPT Neo

If you're retraining a model on new text, you need to download and load the GPT-2 model into the GPU. 

There are several sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M` (default): the "medium" model, 1.5GB on disk.
* `774M` (default): the "large" model, 3GB on disk.

You can also finetune a GPT Neo model instead, which is more suitable for longer texts and the base model has more recent data:

* `125M`: Analogous to the GPT-2 124M model.
* `350M`: Analogous to the GPT-2 355M model

The next cell downloads the model and saves it in the Colaboratory VM. If the model has already been downloaded, running this cell will reload it.

In [ ]:
model='124M'
#model='355M'
#model='774M'

#model='gpt-neo-125M'
#model='gpt-neo-350M'

In [ ]:
if load_model is None:
    if model == '124M' or model == '355M' or model == '774M':
        ai = aitextgen.aitextgen(tf_gpt2=model, to_gpu=True)
    else:
        ai = aitextgen.aitextgen(model='EleutherAI/' + model, to_gpu=True)

## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/w3wvHhR.png)

Upload **any smaller text file** (for example, [a text file of Shakespeare plays](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt)) and update the file name in the cell below, then run the cell.

In [ ]:
data_root_dir = gdrive_root_dir + '/aitextgen/training_data'
datasets = ['meta_reports_combined']
dataset_splits = [1]
dataset_iterations = [200]
dataset_runs = [20]
dataset_learnrates = [1e-3]
save_partial = True

file_basename = 'dataset_cache'
file_ext = '.tar.gz'
from_cache = True

If your text file is large (>10MB), it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

Additionally, you may want to consider [compressing the dataset to a cache first](https://docs.aitextgen.io/dataset/) on your local computer, then uploading the resulting `dataset_cache.tar.gz` and setting the `file_name`in the previous cell to that.

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2 in aitextgen. It runs for `num_steps`, and a progress bar will appear to show training progress, current loss (the lower the better the model), and average loss (to give a sense on loss trajectory).

The model will be saved every `save_every` steps in `trained_model` by default, and when training completes. If you mounted your Google Drive, the model will _also_ be saved there in a unique folder.

The training might time out after 4ish hours; if you did not mount to Google Drive, make sure you end training and save the results so you don't lose them! (if this happens frequently, you may want to consider using [Colab Pro](https://colab.research.google.com/signup))

Important parameters for `train()`:

- **`line_by_line`**: Set this to `True` if the input text file is a single-column CSV, with one record per row. aitextgen will automatically process it optimally.
- **`from_cache`**: If you compressed your dataset locally (as noted in the previous section) and are using that cache file, set this to `True`.
- **`num_steps`**: Number of steps to train the model for.
- **`generate_every`**: Interval of steps to generate example text from the model; good for qualitatively validating training.
- **`save_every`**: Interval of steps to save the model: the model will be saved in the VM to `/trained_model`.
- **`save_gdrive`**: Set this to `True` to copy the model to a unique folder in your Google Drive, if you have mounted it in the earlier cells
- **`fp16`**: Enables half-precision training for faster/more memory-efficient training. Only works on a T4 or V100 GPU.

Here are other important parameters for `train()` that are useful but you likely do not need to change.

- **`learning_rate`**: Learning rate of the model training.
- **`batch_size`**: Batch size of the model training; setting it too high will cause the GPU to go OOM. (if using `fp16`, you can increase the batch size more safely)

In [ ]:
model_root_dir = gdrive_root_dir + '/aitextgen/models/' + run_id

if save_partial:
    model_partial_dir = model_root_dir + '/partial'
    partial_models = []

In [ ]:
for i in range(len(datasets)):
    file_basepath = data_root_dir + '/' + datasets[i] + '/' + file_basename

    for j in range(dataset_runs[i]):
        if save_partial and (i < len(datasets) - 1 or j < dataset_runs[i] - 1):
            partial_model_name = datasets[i] + '_trainrun-' + str(j + 1)
            model_dir = model_partial_dir + '/' + partial_model_name
            partial_models.append(partial_model_name)
        else:
            model_dir = model_root_dir

        for k in range(dataset_splits[i]):
            if dataset_splits[i] > 1:
                current_file = file_basepath + '.' + str(k) + file_ext
            else:
                current_file = file_basepath + file_ext
            ai.train(current_file,
                     line_by_line=False,
                     from_cache=from_cache,
                     num_steps=dataset_iterations[i],
                     generate_every=dataset_iterations[i],
                     save_every=dataset_iterations[i],
                     save_gdrive=False,
                     run_id=run_id,
                     output_dir=model_dir,
                     learning_rate=dataset_learnrates[i],
                     fp16=False,
                     batch_size=1)

            # R.B.: required to prevent memory leaks in Colab
            gc.collect()

27/05/2022 12:42:11 — INFO — aitextgen — Loading text from /content/drive/My Drive/aitextgen/training_data/meta_reports_combined/dataset_cache.tar.gz with generation length of 1024.
27/05/2022 12:42:11 — INFO — aitextgen.TokenDataset — TokenDataset containing 91,415 subsets loaded via cache.
27/05/2022 12:42:11 — INFO — torch.distributed.nn.jit.instantiator — Created a temporary directory at /tmp/tmp905iag67
27/05/2022 12:42:11 — INFO — torch.distributed.nn.jit.instantiator — Writing /tmp/tmp905iag67/_remote_module_non_sriptable.py
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors

  0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:2265: LightningDeprecationWarning: `trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7. Use `ProgressBarBase.get_metrics` instead.
  "`trainer.progress_bar_dict` is deprecated in v1.5 and will be removed in v1.7."


200 steps reached: saving model to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-1
200 steps reached: generating sample texts.
 the
completeness System (REMK)

3.3

ended

infoidices

As all other persons see it, they are considered important to discuss their findings with the
web site within the
Spy user.

eta should be aware that they are able to
be able to view the data they have received from the company’s
facts and protect other data.

3.3.3

 position

info@ cybersleuthlab.org

https://www. cybersleuthlab.org

During the analysis process, the examiner should ensure that
no data is filtered and
not only the
data they send from the provider’s
data.

3.3

Digital Forensics Guide

Date
https://www. cybersleuthlab.org

http://www.cybersleuthlab.org

igh
http://www.cybersleuthlab.org

http://www.cybersleuthlab.org

http://www.cybersleuthlab.org

http://www.cybersleuthlab.org




27/05/2022 12:43:33 — INFO — aitextgen — Saving trained model pytorch_model.bin to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-1
27/05/2022 12:43:35 — INFO — aitextgen — Loading text from /content/drive/My Drive/aitextgen/training_data/meta_reports_combined/dataset_cache.tar.gz with generation length of 1024.
27/05/2022 12:43:35 — INFO — aitextgen.TokenDataset — TokenDataset containing 91,415 subsets loaded via cache.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

200 steps reached: saving model to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-19
200 steps reached: generating sample texts.

FULL FOOTAGE: Intact Fetuses "Just a Matter of Line
Items" for Planned Parenthood TX Mega-Center
Begins 2015.04.09 07:37:48 (034977)
Key fingerprint = AF19 FA27 2F94 998D FDB5 DE3D F8B5 06E4 A169 4E46

The side, HDD and Floppy drive removed the system,
Physical and photographed a rest of the system

Investigation Serial number
: REC/SJA/01Aug04.AVI

The system as discovered minus the HDD was described and catalogued as:

05
,A

Case and Motherboard testing

ut

ho

rr

Investigation Serial number
: REC/SJA/01Aug04.AVI

Description:
Epic tower.com

©

SA

NS

In

sti

tu

te

20

Key fingerprint = AF19 FA27 2F94 998D FDB5 DE3D F8B5 06E4 A169 4E46

8 of 57
© SANS Institute 2005


27/05/2022 13:10:51 — INFO — aitextgen — Saving trained model pytorch_model.bin to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-19
27/05/2022 13:10:53 — INFO — aitextgen — Loading text from /content/drive/My Drive/aitextgen/training_data/meta_reports_combined/dataset_cache.tar.gz with generation length of 1024.
27/05/2022 13:10:53 — INFO — aitextgen.TokenDataset — TokenDataset containing 91,415 subsets loaded via cache.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/200 [00:00<?, ?it/s]

200 steps reached: saving model to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47
200 steps reached: generating sample texts.


©

SA

NS

In

sti

tu
te

20

Key fingerprint = AF19 FA27 2F94 998D FDB5 DE3D F8B5 06E4 A169 4E46

© SANS

The purpose of the samba share was to allow the use of windows based tools (with
which the investigator was more familiar). Upon connecting to the read only
network share, a complete copy of the contents was taken to a temporary directory
E:\GIAC SANS TEMP\.DEFAULT_ARMS
Output
Volume Serial Number was compared to the original dcfldd command_hdd_sda1_dd: x86 boot sector, code
offset0x58, OEM-ID “MSWIN4.0"

Aug date: 7/25/2014, and this was compared with a
GPRS software log. However, research on the role of thensics Digital Imaging Technical is to
examine the contents of the Council of Europe/Ash exam.... is in a particular case.

1.
2.

tu

te

20

The function


27/05/2022 13:12:22 — INFO — aitextgen — Saving trained model pytorch_model.bin to //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47


You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text.

**If you just trained a model**, you'll get much faster training performance if you reload the model; the next cell will reload the model you just trained from the `trained_model` folder.

`generate()` without any parameters generates a single text from the loaded model to the console.

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = ai.generate_one()`

You can also pass in a `prompt` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `n`. You can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 50 for `batch_size` to avoid going OOM).

Other optional-but-helpful parameters for `ai.generate()` and friends:

*  **`min length`**: The minimum length of the generated text: if the text is shorter than this value after cleanup, aitextgen will generate another one.
*  **`max_length`**: Number of tokens to generate (default 256, you can generate up to 1024 tokens with GPT-2 and 2048 with GPT Neo)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

For bulk generation, you can generate a large amount of texts to a file and sort out the samples locally on your computer. The next cell will generate `num_files` files, each with `n` texts and whatever other parameters you would pass to `generate()`. The files can then be downloaded from the Files sidebar!

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
num_outputs = 5
max_length = 1000
temperature = 1.0
top_p = 0.9

prompts = ['Digital Forensics Analysis Report\n',
           'This report is ',
           'The contents of ',
           'Conclusion\n',
           'It is recommended that ',
           'In the opinion of the expert, ',
           'File \'Exploit_Office\' contains ',
           'File \'Exploit_Office\' does not contain ',
           'Website \'Webmail SquirrelMail\' contains ',
           'Website \'Webmail SquirrelMail\' does not contain ',
           'Bill Due to past contains a link \'https://genom.mefst.hr/webmail/src/login.php\' to a website \'Webmail SquirrelMail\'.',
           'New Dogecoin Crypto Sale contains a link \'http://webmail.forumofthemall.hr/mail/loging.php\' to a website \'Webmail SquirrelMail Popular Forum\'.',
           'New OneCoin Crypto Sale contains a link \'http://',
           'Note of eviction contains ',
           'Note of eviction contains attachment ',
           'Note of eviction contains attachment \'Exploit_Office\'. Attachment is quarantined on \'Mail server EP\'.',
           'Log entry found: ',
           'Log entry found: Firewall (Type: Firewall) ',
           'Log entry found: Firewall (Type: Firewall) detected. [Allowed network traffic protocol ',
           'Log entry found: Firewall (Type: Firewall) blocked. [Blocked network traffic protocol ',
           'Log entry found: Firewall (Type: Firewall) detected. [Allowed network traffic protocol \'smtp:25\' from \'server74.aws.com\' to \'Mail server EP\'. Rule \'Internet_to_Mail_Server\'.]',
           'Log entry found: Firewall (Type: Firewall) detected. [Allowed network traffic protocol \'https:443\' from \'Proxy server\' to \'server74.aws.com\'. Rule \'Proxy_to_Internet, https:443\'.]',
           'Log entry found: Firewall (Type: Firewall) blocked. [Blocked network traffic protocol \'https:443\' from \'PCSZT03\' to \'Firewall TSO Enterprise\'.]',
           'Log analysis on ',
           'Log analysis on \'Firewall TSO Enterprise\' for period 1.1.2022. 0:00:00 - 4.2.2022. 13:50:44 finished. Report is ready.']

In [ ]:
generate_partial_outputs = True

num_outputs_partial = 2
max_length_partial = 500
prompts_partial = ['Digital Forensics Analysis Report\n',
                   'This report is ',
                   'The contents of ',
                   'Conclusion\n']

In [ ]:
output_root_dir = gdrive_root_dir + '/aitextgen/outputs/' + run_id
output_ext = '.txt'

if not os.path.exists(output_root_dir):
    os.makedirs(output_root_dir)

In [ ]:
if save_partial and generate_partial_outputs:
    output_partial_dir = output_root_dir + '/partial'

    for i in partial_models:
        model_dir = model_partial_dir + '/' + i
        ai = aitextgen.aitextgen(model_folder=model_dir, to_gpu=True)

        output_dir = output_partial_dir + '/' + i
        partial_id = run_id + '_partial_' + i
        output_basepath = output_dir + '/' + partial_id + '_output'

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        for j in range(len(prompts_partial)):
            if len(prompts_partial) > 1:
                current_output = output_basepath + '.' + str(j) + output_ext
            else:
                current_output = output_basepath + output_ext
            ai.generate_to_file(n=num_outputs_partial,
                                batch_size=1,
                                prompt=prompts_partial[j],
                                max_length=max_length_partial,
                                temperature=temperature,
                                top_p=top_p,
                                destination_path=current_output)

27/05/2022 13:12:24 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-1.
27/05/2022 13:12:29 — INFO — aitextgen — GPT2 loaded with 124M parameters.
27/05/2022 13:12:29 — INFO — aitextgen — Using the default GPT-2 Tokenizer.
27/05/2022 13:12:30 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-1/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_partial_meta_reports_combined_trainrun-1_output.0.txt


  0%|          | 0/2 [00:00<?, ?it/s]

27/05/2022 13:12:42 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-1/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_partial_meta_reports_combined_trainrun-1_output.1.txt


  0%|          | 0/2 [00:00<?, ?it/s]

27/05/2022 13:12:52 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-1/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_partial_meta_reports_combined_trainrun-1_output.2.txt


...

  0%|          | 0/2 [00:00<?, ?it/s]

27/05/2022 13:26:53 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-19/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_partial_meta_reports_combined_trainrun-19_output.2.txt


  0%|          | 0/2 [00:00<?, ?it/s]

27/05/2022 13:27:04 — INFO — aitextgen — Generating 2 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/partial/meta_reports_combined_trainrun-19/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_partial_meta_reports_combined_trainrun-19_output.3.txt


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
if len(datasets) > 0:
    ai = aitextgen.aitextgen(model_folder=model_root_dir, to_gpu=True)

output_basepath = output_root_dir + '/' + run_id + '_output'

for i in range(len(prompts)):
    if len(prompts) > 1:
        current_output = output_basepath + '.' + str(i) + output_ext
    else:
        current_output = output_basepath + output_ext
    ai.generate_to_file(n=num_outputs,
                        batch_size=1,
                        prompt=prompts[i],
                        max_length=max_length,
                        temperature=temperature,
                        top_p=top_p,
                        destination_path=current_output)

27/05/2022 13:27:14 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/My Drive/aitextgen/models/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47.
27/05/2022 13:27:20 — INFO — aitextgen — GPT2 loaded with 124M parameters.
27/05/2022 13:27:20 — INFO — aitextgen — Using the default GPT-2 Tokenizer.
27/05/2022 13:27:20 — INFO — aitextgen — Generating 5 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_output.0.txt


  0%|          | 0/5 [00:00<?, ?it/s]

27/05/2022 13:28:10 — INFO — aitextgen — Generating 5 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_output.1.txt


  0%|          | 0/5 [00:00<?, ?it/s]

27/05/2022 13:29:00 — INFO — aitextgen — Generating 5 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_output.2.txt


...

  0%|          | 0/5 [00:00<?, ?it/s]

27/05/2022 13:46:20 — INFO — aitextgen — Generating 5 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_output.23.txt


  0%|          | 0/5 [00:00<?, ?it/s]

27/05/2022 13:47:10 — INFO — aitextgen — Generating 5 texts to /content/drive/My Drive/aitextgen/outputs/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47/aitextgen-CCS-124M-2000-lr3_run_2022-05-27-12-41-47_output.24.txt


  0%|          | 0/5 [00:00<?, ?it/s]

# LICENSE

MIT License

Copyright (c) 2020-2021 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.